In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm321 (
	hash_id BIGINT
	,recordmode STRING
	,kappl STRING
	,kschl STRING
	,vkorg STRING
	,vtweg STRING
	,werks STRING
	,bbynr STRING
	,matnr STRING
	,vrkme STRING
	,datbi DATE
	,orgty STRING
	,datab DATE
	,knumh STRING
	,vakey STRING
	,rewref STRING
	,bbyval DOUBLE
	,bbyper DOUBLE
	,bbycur STRING
	,kpein DOUBLE
	,kmein STRING
	,grpnr STRING
	,stfkz STRING
	,mquan DOUBLE
	,bbytype STRING
	,filgr STRING
	,proc_date DATE,
	file_creation_ts TIMESTAMP
	)
	TBLPROPERTIES 
	(
		DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE = TRUE,
		DELTA.AUTOOPTIMIZE.AUTOCOMPACT = TRUE
	)
""")

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_vccmdm32',
    proc_date,
    'temp_sap_avccmdm321'
)

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW sap_avccmdm321 
AS
WITH temp AS (
SELECT  reqtsn
        ,datapakid
        ,record
        ,recordmode	
        ,IF(kappl  = '',null,kappl ) kappl
        ,IF(kschl  = '',null,kschl ) kschl
        ,IF(vkorg  = '',null,vkorg ) vkorg
        ,IF(vtweg  = '',null,vtweg ) vtweg
        ,IF(werks  = '',null,werks ) werks
        ,IF(bbynr  = '',null,LTRIM('0',bbynr) ) bbynr
        ,IF(matnr  = '',null,LTRIM('0',matnr) ) matnr
        ,IF(vrkme  = '',null,vrkme ) vrkme
        ,TO_DATE(IF(datbi  = '',null,datbi),'yyyyMMdd') datbi
        ,IF(orgty  = '',null,orgty ) orgty
        ,TO_DATE(IF(datab  = '',null,datab ),'yyyyMMdd') datab
        ,IF(knumh  = '',null,knumh ) knumh
        ,IF(vakey  = '',null,LTRIM('0',vakey) ) vakey
        ,IF(rewref = '',null,rewref) rewref
        ,bbyval
        ,bbyper
        ,IF(bbycur = '',null,bbycur) bbycur
        ,kpein
        ,IF(kmein = '',null,kmein) kmein
        ,IF(grpnr = '',null,grpnr) grpnr
        ,IF(stfkz = '',null,stfkz) stfkz
        ,mquan
        ,IF(bbytype = '',null,bbytype) bbytype
        ,IF(filgr   = '',null,filgr  ) filgr
        ,proc_date
        ,file_creation_ts
         FROM (
            SELECT * EXCEPT(rn) FROM (
                SELECT * , ROW_NUMBER() OVER (PARTITION BY reqtsn, datapakid, record ORDER BY proc_date DESC) rn FROM  temp_sap_avccmdm321
            ) a WHERE rn = 1 
        ) a
),
main AS (
SELECT FARM_FINGERPRINT(CONCAT(
    IFNULL(CAST(kappl AS STRING), ""),
    IFNULL(CAST(kschl AS STRING), ""),
    IFNULL(CAST(vkorg AS STRING), ""),
    IFNULL(CAST(vtweg AS STRING), ""),
    IFNULL(CAST(werks AS STRING), ""),
    IFNULL(CAST(bbynr AS STRING), ""),
    IFNULL(CAST(matnr AS STRING), ""),
    IFNULL(CAST(vrkme AS STRING), ""),
    IFNULL(CAST(datbi AS STRING), "")
)) hash_id,
* FROM temp
),
main_2 AS (
	SELECT * 
	, ROW_NUMBER() OVER (PARTITION BY hash_id ORDER BY reqtsn DESC,datapakid DESC,record  DESC) rn 
		FROM main
)
SELECT 
* EXCEPT(rn,reqtsn,datapakid,record) FROM main_2 WHERE rn = 1
""")

In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm321 main_tbl
USING sap_avccmdm321 temp_tbl
ON main_tbl.hash_id = temp_tbl.hash_id 
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;
""")